# Boltzmann Machines

## Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import torch #
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

## Importing the dataset

In [4]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::' , header = None , engine = 'python' , encoding = 'latin-1' )
users = pd.read_csv('ml-1m/users.dat', sep='::' , header = None , engine = 'python' , encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::' , header = None , engine = 'python' , encoding = 'latin-1')

NameError: name 'pd' is not defined

## Preparing the training set and the test set

In [ ]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test' , delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies

In [ ]:
nb_users = max(max(training_set[:,0]), max(test_set[:,0]))
nb_movies = max(max(training_set[:,1]), max(test_set[:,1]))
print(nb_users)
print(nb_movies)

## Converting the data into an array with users in lines and movies in columns

In [ ]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:, 0] == id_users]
        id_ratings = data[:,2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings [id_movies -1] = id_ratings
        new_data.append(list(ratings))

    return new_data

training_set = convert(training_set)
test_set = convert(test_set)



## Converting the data into Torch tensors

In [ ]:
#Torch tensors çok boyutlu dizilerdir(MATRIX).
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [ ]:
training_set[training_set == 0] = -1 #filmi hiç izlemediyse -1 yaparız ve ağırlığa dahil edilmez.
training_set[training_set == 1] = 0 #puanı 1 veya 2 olanlar beğenilmedi demek..
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1 #beğenildi demek ..
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network

In [ ]:
class RBM():
    def __init__(self, nv, nh): #numbervisible, numberhidden
      self.W = torch.randn(nh, nv) #ağırlık matrisini rastgele başlattık .
      self.a = torch.randn(1, nh)
      self.b = torch.randn(1, nv)


    def sample_h(self, x): #x görünür katmanın girdisi.
      wx = torch.mm(x, self.W.t()) #gizli nöronlara ulaşa toplam girdiler = wx
      activation = wx + self.a.expand_as(wx) #gizli katmanda aktivasyon hesaplanır.
      p_h_given_v = torch.sigmoid(activation) #sigmoid fonksiyonu ile aktivasyon olasılığa dönüşür.
      return p_h_given_v, torch.bernoulli(p_h_given_v) # Bernoulli dağılımına göre rastgele 0 veya 1 değerleri üreterek gizli katmanda hangi nöronların aktif olacağını belirliyoruz.

    def sample_v(self, y):
      wy = torch.mm(y, self.w)
      activation = wy + self.b.expand_as(wy)
      p_v_given_h = torch.sigmoid(activation)
      return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
      self.w += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
      self.b += torch.sum((v0-vk),0)
      self.a += torch.sum((ph0-phk),0)

nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)  #creating object.






## Training the RBM

In [ ]:
#RBM'i Contrastive Divergence ile eğitiyoruz.
nb_epoch = 10
for epoch in range(1, nb_epoch + 1): #Süreç 10 defa tekrarlanır...
  train_loss = 0 # Eğitim kaybını sıfırlıyoruz.
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user:id_user+batch_size] # güncellenen versioyn.
    v0 = training_set[id_user:id_user+batch_size] #orijinal girişler
    ph0,_ = rbm.sample_h(v0) #gizli katmandaki aktivasyon olasılıklarını hesaplıyoruz.

    #k=10 iterasyon boyunca (Contrastive Divergence için yeterli) bu işlemi tekrarlıyoruz.
    #İlk olarak: vk'dan gizli katmanı hesaplıyoruz (sample_h).
    #Sonra: h'den tekrar görünür katmana geçiyoruz (sample_v).
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]

    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
     #RMSE here
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


## Testing the RBM

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        #RMSE
        s += 1.
print('test loss: '+str(test_loss/s))

Recommending Movies to User

In [ ]:
def recommend_movies(user_id, model, training_set, nb_movies, movies_df, top_n=5):
    user_data = training_set[user_id - 1].unsqueeze(0)  # Kullanıcı verisini al
    _, hidden = model.sample_h(user_data)  # Gizli katman aktivasyonlarını al
    predicted_ratings, _ = model.sample_v(hidden)  # Tahmin edilen puanları al

    predicted_ratings = predicted_ratings.data.numpy().flatten()  # NumPy dizisine çevir

    watched_movies = training_set[user_id - 1] > 0  # Kullanıcının izlediği filmler
    predicted_ratings[watched_movies] = -np.inf  # Zaten izlenen filmleri önerme

    top_movies_idx = predicted_ratings.argsort()[-top_n:][::-1]  # En yüksek tahmin edilen filmleri al

    recommended_movies = movies_df.iloc[top_movies_idx]  # Filmleri movies.csv'den çek

    return recommended_movies

# Örnek kullanım (kullanıcı 10 için)
recommended_movies = recommend_movies(10, rbm, training_set, nb_movies, movies)
print(recommended_movies)
